# data cleaning:
# # [FAO] production of crops and livestock products

In [8]:
import pandas as pd

import _functions_sql as fs
import _functions_data_files as fdf

source_dir = 'fao_production'

## import from CSV & general overview

In [9]:
# define import csv
file_name = 'Production_Crops_Livestock_E_All_Data_(Normalized).csv'

# import raw table
df_raw = pd.read_csv(
      fdf.get_path(file_name, source_dir)
    , encoding='latin-1'
    , converters={'Note': str} # handle DtypeWarning without 'low_memory=False' 
)

In [10]:
df_raw.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1961,1961,ha,0.0,A,
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1962,1962,ha,0.0,A,
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1963,1963,ha,0.0,A,
3,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1964,1964,ha,0.0,A,
4,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1965,1965,ha,0.0,A,


In [11]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4127584 entries, 0 to 4127583
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Area Code        int64  
 1   Area Code (M49)  object 
 2   Area             object 
 3   Item Code        int64  
 4   Item Code (CPC)  object 
 5   Item             object 
 6   Element Code     int64  
 7   Element          object 
 8   Year Code        int64  
 9   Year             int64  
 10  Unit             object 
 11  Value            float64
 12  Flag             object 
 13  Note             object 
dtypes: float64(1), int64(5), object(8)
memory usage: 440.9+ MB


In [12]:
# show time span
df_raw['Year'].unique()


array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [13]:
# show full duplicates
df_raw.duplicated().value_counts()

False    4127584
Name: count, dtype: int64

In [14]:
# show null values
df_raw.isnull().value_counts()

Area Code  Area Code (M49)  Area   Item Code  Item Code (CPC)  Item   Element Code  Element  Year Code  Year   Unit   Value  Flag   Note 
False      False            False  False      False            False  False         False    False      False  False  False  False  False    4127584
Name: count, dtype: int64

## drop columns

### drop 'Area Code (M49)' & 'Area' (redundant to 'Area Code')

In [ ]:
# import additional table for AreaCodes (provided by the same download zip)
file_name = 'Production_Crops_Livestock_E_AreaCodes.csv'
df_prod_areacodes = pd.read_csv(
      fdf.get_file_path(file_name, source_dir)
    , encoding='latin-1' # content is not UTF-8 compatible :TODO: :FIXME: ???
)

In [ ]:
df_prod_areacodes.head()

,Area Code,M49 Code,Area
0,2,'004,Afghanistan
1,5100,'002,Africa
2,3,'008,Albania
3,4,'012,Algeria
4,5200,'019,Americas


In [ ]:
# verify no duplicated rows are present
df_prod_areacodes.duplicated(keep='first').sum()

0

In [ ]:
# rename columns to perfectly match during merge
df_prod_areacodes.rename(columns={
    'M49 Code': 'Area Code (M49)'
}, inplace=True)

In [ ]:
# left merge on 'df_prod' to inspect potentially redundant columns
df_prod_temp = df_prod.merge(df_prod_areacodes, how='left', on='Area Code')

In [ ]:
df_prod_temp.head()

,Area Code,Area Code (M49)_x,Area_x,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Area Code (M49)_y,Area_y
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1961,1961,ha,0.0,A,,'004,Afghanistan
1,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1962,1962,ha,0.0,A,,'004,Afghanistan
2,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1963,1963,ha,0.0,A,,'004,Afghanistan
3,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1964,1964,ha,0.0,A,,'004,Afghanistan
4,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1965,1965,ha,0.0,A,,'004,Afghanistan


In [ ]:
# count rows with divergent M49 area codes
df_prod_temp.query(
  '`Area Code (M49)_x` != `Area Code (M49)_y`'
).shape[0]

0

In [ ]:
# drop 'Area Code (M49)' column as it contains no additional info
df_prod = df_prod.drop(columns = ['Area Code (M49)'])	

In [ ]:
# show divergent area values before and after merge
pd.DataFrame(pd.unique(df_prod_temp.query(
    '`Area_x` != `Area_y`'
)[['Area_x', 'Area_y']].values.ravel()))

,0
0,"China, Hong Kong SAR"
1,China; Hong Kong SAR
2,"China, Macao SAR"
3,China; Macao SAR
4,"China, mainland"
5,China; mainland
6,"China, Taiwan Province of"
7,China; Taiwan Province of


In [ ]:
# compare divergent area values after replacing ',' with ';'
df_prod_temp['Area_y'] = df_prod_temp['Area_y'].str.replace(',', ';')
df_prod_temp['is_equal'] = df_prod_temp['Area_x'] == df_prod_temp['Area_y']

print(df_prod_temp)

KeyError: 'Area_y'

In [ ]:
# drop 'Area' column as it contains no additional info
# :FIXME: :TODO: ommitted for transparency in sql tables
# df_prod = df_prod.drop(columns = ['Area'])

### drop 'Item Code' & 'Item Code (CPC)' (redundant to 'Item')

In [ ]:
# import additional table for ItemCodes (provided by the same download zip)
file_name = 'Production_Crops_Livestock_E_ItemCodes.csv'
df_prod_itemcodes = pd.read_csv(
      fdf.get_file_path(file_name, source_dir)
    , encoding='latin-1' # content is not UTF-8 compatible :TODO: :FIXME: ???
)

In [ ]:
df_prod_itemcodes.head()

,Item Code,CPC Code,Item
0,101,'01195,Canary seed
1,1016,'02123,Goats
2,1017,'21116,Meat of goat; fresh or chilled
3,1018,'21156,Edible offal of goat; fresh; chilled or frozen
4,1019,'21515,Goat fat; unrendered


In [ ]:
# verify no duplicated rows are present
df_prod_itemcodes.duplicated(keep='first').sum()

0

In [ ]:
# rename columns to perfectly match during merge
df_prod_itemcodes.rename(columns={
    'CPC Code': 'Item Code (CPC)'
}, inplace=True)

In [ ]:
# left merge on 'df_prod' to inspect potentially redundant 'Item Code' columns
df_prod_temp = df_prod.merge(df_prod_itemcodes, how='left', on='Item Code')

In [ ]:
df_prod_temp.head()

,Area Code,Area,Item Code,Item Code (CPC)_x,Item_x,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note,Item Code (CPC)_y,Item_y
0,2,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1961,1961,ha,0.0,A,,'01371,Almonds; in shell
1,2,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1962,1962,ha,0.0,A,,'01371,Almonds; in shell
2,2,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1963,1963,ha,0.0,A,,'01371,Almonds; in shell
3,2,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1964,1964,ha,0.0,A,,'01371,Almonds; in shell
4,2,Afghanistan,221,'01371,"Almonds, in shell",5312,Area harvested,1965,1965,ha,0.0,A,,'01371,Almonds; in shell


In [ ]:
# count rows with divergent CPC item codes
df_prod_temp.query(
  '`Item Code (CPC)_x` != `Item Code (CPC)_y`'
).shape[0]

0

In [ ]:
# drop 'Item Code (CPC)' column as it contains no additional info
df_prod = df_prod.drop(columns = ['Item Code (CPC)'])	

In [ ]:
# show divergent item values before and after merge
pd.DataFrame(pd.unique(df_prod_temp.query(
    '`Item_x` != `Item_y`'
)[['Item_x', 'Item_y']].values.ravel()))

,0
0,"Almonds, in shell"
1,Almonds; in shell
2,"Anise, badian, coriander, cumin, caraway, fenn..."
3,Anise; badian; coriander; cumin; caraway; fenn...
4,"Buttermilk, dry"
...,...
243,Abaca; manila hemp; raw
244,"Snails, fresh, chilled, frozen, dried, salted ..."
245,Snails; fresh; chilled; frozen; dried; salted ...
246,"Kapok fibre, raw"


In [ ]:
# count rows with divergent item codes
df_prod_temp.query(
      '`Item Code_x` != `Item Code_y` \
    or `Item Code (CPC)_x` != `Item Code (CPC)_y`'
).shape[0]

1880725

In [ ]:
# count rows with divergent item codes except those, where 'ItemCodes.csv'
# lacks an entry
df_prod_temp.query(
      '`Item Code_x` != `Item Code_y` and `Item Code_y`.notnull() \
    or \
           `Item Code (CPC)_x` != `Item Code (CPC)_y` \
       and `Item Code (CPC)_y`.notnull()'
).shape[0]

0

:TODO: :FIXME: inspect which rows are affected by missing values!

In [ ]:
# drop the 'Item Code' and 'Item Code (CPC)' columns as they contain no
# additional info other then 'Item'
df_prod = df_prod.drop(columns = ['Item Code', 'Item Code (CPC)'])	

### drop 'Element Code' (redundant to 'Element')

In [ ]:
df_prod[['Element Code', 'Element']].drop_duplicates()

,Element Code,Element
0,5312,Area harvested
62,5419,Yield
109,5510,Production
704,5111,Stocks
1458,5320,Producing Animals/Slaughtered
1642,5112,Stocks
3127,5410,Yield
3189,5413,Yield
3313,5513,Production
3375,5313,Laying


In [ ]:
df_prod[['Element Code']].drop_duplicates().shape[0]

18

In [ ]:
df_prod[['Element']].drop_duplicates().shape[0]

9

:FIXME: :TODO: can not be dropped, since not equals...

### drop 'Year Code' column (redundant to 'Year') 

In [17]:
# show divergent rows
df_raw.query('`Year Code` != `Year`').shape[0]

0

In [19]:
# drop column as it contains the same values as 'Year'
df_raw = df_raw.drop(columns = 'Year Code')

### drop 'Note' column (no value)

In [21]:
# show unique values
df_raw['Note'].unique()

array(['', 'Unofficial figure'], dtype=object)

In [23]:
# drop it as it doescontains no relevant info
df_raw = df_raw.drop(columns = 'Note')

## rename columns

In [24]:
# Rename remaining columns
df_raw.rename(columns={
      'Area': 'area'
    , 'Item': 'item'
    , 'Element Code': 'element_code'
    , 'Element': 'element'
    , 'Year': 'year'
    , 'Unit': 'unit'
    , 'Value': 'value'
    , 'Flag': 'flag'
}, inplace=True)

### verify, column 'Unit' has comparable units of measurement

In [26]:
df_raw[['unit']].drop_duplicates()

,unit
0,ha
62,100 g/ha
109,t
704,An
1642,1000 An
3127,100 mg/An
3189,No/An
3313,1000 No
3871,100 g/An
4243,0.1 g/An


:FIXME: :TODO: comparison for the following needed: 'An' + '1000 An', '100mg/An' + '100g/An' + '0.1g/An', 'No' + '1000 No'

### add 'Flag' values from additional table

:FIXME: :TODO: tbd!!!!!

# final output

In [ ]:
df_prod.head()

,area,item,element_code,element,year,unit,value,flag
0,Afghanistan,"Almonds, in shell",5312,Area harvested,1961,ha,0.0,A
1,Afghanistan,"Almonds, in shell",5312,Area harvested,1962,ha,0.0,A
2,Afghanistan,"Almonds, in shell",5312,Area harvested,1963,ha,0.0,A
3,Afghanistan,"Almonds, in shell",5312,Area harvested,1964,ha,0.0,A
4,Afghanistan,"Almonds, in shell",5312,Area harvested,1965,ha,0.0,A


In [ ]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4127584 entries, 0 to 4127583
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   area          object 
 1   item          object 
 2   element_code  int64  
 3   element       object 
 4   year          int64  
 5   unit          object 
 6   value         float64
 7   flag          object 
dtypes: float64(1), int64(2), object(5)
memory usage: 251.9+ MB


# hmmmm

Exploring the element column, we see that not all values are relevant for further analysis. According to element definitions, yield is the production per unit of harvested area 

In [ ]:
#df_prod["Element"].value_counts()

# export to database

In [ ]:
# write cleaned data into database
fs.write_dataframe(df_raw, 'fao_production_sh')

+ table written: fao_production_sh
